# Automated ML

In [1]:
import azureml.core
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import Model
from sklearn.model_selection import train_test_split
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.core.webservice import LocalWebservice
from azureml.core.webservice import Webservice, AciWebservice
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import requests
import json

## Setup

In [2]:
ws = Workspace.from_config()
experiment_name = 'capstone'
experiment=Experiment(ws, experiment_name)

In [3]:
amlcompute_cluster_name = "capstonecompute"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)


Creating
Succeeded...............................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
The dataset used for the capstone project is the Heart Failure Clinical Records dataset that can be found [here](https://archive.ics.uci.edu/ml/datasets/Heart+failure+clinical+records) at the UCI Machine Learning Repository.  The dataset will be used for classification to help predict a death event.

In [4]:
# test to see if dataset is in store
key = 'heartfailuredataset'
if key in ws.datasets.keys(): 
    dataset = ws.datasets[key] 
    print('Found dataset')

# if not, load the dataset, save it to the store
else:
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(url)
    dataset = dataset.register(workspace=ws, name=key)
    
               


In [5]:
dataset.to_pandas_dataframe().head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [6]:
# split into data and test sets
ds = dataset.to_pandas_dataframe()
# use train test split
train, test = train_test_split(ds, random_state=625, shuffle=True)

# then write the train data to a file and load to datastore
train.to_csv('train.csv', index=False)
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

# then read to tabular dataset for automl
train = Dataset.Tabular.from_delimited_files(path = [(datastore,'train.csv')])


Uploading an estimated of 1 files
Uploading ./train.csv
Uploaded ./train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

To reduce the time taken to train, experiment_timeout_minutes of 30 and 50 iterations were chosen.  In order to utilize more than one node at once, since 4 where allocated, max_concurrent_iterations were set to 4.  Early stopping is enabled so if a run is not performing well, it can stop early, again to save time and if not performing well continuing seems uncessary.  The local compute was chosen as this may be slower but generally provides better results.  Also 5 cross validations were chosen to split the data into 80%/20% train and test sets and help balance variance and bias.

In [7]:
automl_settings = {
    "experiment_timeout_minutes":30,
    "enable_early_stopping":True,    
    "primary_metric":'accuracy',
    "n_cross_validations":5,
    "iterations":50,
    "max_concurrent_iterations":4,
}

automl_config = AutoMLConfig(
    compute_target=compute_target, 
    task='classification', 
    training_data=train,
    label_column_name='DEATH_EVENT',
    **automl_settings)

In [8]:
run = experiment.submit(automl_config)

Running on remote.


## Run Details


In [9]:
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [10]:
status = run.wait_for_completion()

## Best Model

In [11]:
best_run, model = run.get_output()

Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


In [12]:
print(best_run)

Run(Experiment: capstone,
Id: AutoML_d3f92056-df67-4e62-a896-ed303c936f5e_36,
Type: azureml.scriptrun,
Status: Completed)


In [13]:
print(model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.056842105263157895,
                                                                                                    min_weight_fraction_leaf=0.0,
                                                                                                    n_estimators=100,
    

In [14]:
best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":4}',
 'ensembled_iterations': '[3, 21, 20, 14, 4, 11, 32]',
 'ensembled_algorithms': "['RandomForest', 'LightGBM', 'GradientBoosting', 'ExtremeRandomTrees', 'RandomForest', 'XGBoostClassifier', 'RandomForest']",
 'ensemble_weights': '[0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.2222222222222222, 0.2222222222222222, 0.1111111111111111]',
 'best_individual_pipeline_score': '0.866060606060606',
 'best_individual_iteration': '3',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}

In [15]:
best_run.get_metrics(name='accuracy')

{'accuracy': 0.8838383838383839}

### Test the model

In [16]:
# Split into x and y tests
y_test = test['DEATH_EVENT']
x_test = test.drop(['DEATH_EVENT'],axis=1)

In [17]:
# test the model and create a confusion matrix
ypred = model.predict(x_test)
cmatrix = confusion_matrix(y_test, ypred)
pd.DataFrame(cmatrix)

,0,1
0,43,6
1,7,19


### Register the model

In [18]:
#registering the model
properties = {"model":"VotingEnsemble","accuracy":str(best_run.get_metrics(name='accuracy'))}
model = best_run.register_model(model_name ='automl_model',model_path="outputs/model.pkl",properties=properties)



In [19]:
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-136002', subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb', resource_group='aml-quickstarts-136002'), name=automl_model, id=automl_model:1, version=1, tags={}, properties={'model': 'VotingEnsemble', 'accuracy': "{'accuracy': 0.8838383838383839}"})


## Model Deployment

### Save the environment

In [20]:
# get the list of environments
Environment.list(workspace=ws).keys()


dict_keys(['AzureML-VowpalWabbit-8.8.0', 'AzureML-AutoML', 'AzureML-Hyperdrive-ForecastDNN', 'AzureML-PyTorch-1.4-CPU', 'AzureML-PyTorch-1.4-GPU', 'AzureML-AutoML-DNN', 'AzureML-AutoML-DNN-GPU', 'AzureML-Designer-Transform', 'AzureML-Designer', 'AzureML-AutoML-GPU', 'AzureML-Designer-IO', 'AzureML-Dask-CPU', 'AzureML-Designer-R', 'AzureML-Dask-GPU', 'AzureML-Designer-Recommender', 'AzureML-PyTorch-1.5-CPU', 'AzureML-PyTorch-1.5-GPU', 'AzureML-Sidecar', 'AzureML-Designer-CV-Transform', 'AzureML-Designer-Score', 'AzureML-Designer-PyTorch', 'AzureML-Designer-CV', 'AzureML-TensorFlow-2.1-GPU', 'AzureML-TensorFlow-2.1-CPU', 'AzureML-Designer-PyTorch-Train', 'AzureML-AutoML-DNN-Vision-GPU', 'AzureML-Designer-VowpalWabbit', 'AzureML-TensorFlow-2.2-GPU', 'AzureML-TensorFlow-2.2-CPU', 'AzureML-PyTorch-1.6-CPU', 'AzureML-PyTorch-1.6-GPU', 'AzureML-Triton', 'AzureML-TensorFlow-2.3-CPU', 'AzureML-TensorFlow-2.3-GPU', 'AzureML-DeepSpeed-0.3-GPU', 'AzureML-TensorFlow-2.0-CPU', 'AzureML-Chainer-5.1.0

In [21]:
# save the environment
my_env = Environment.get(workspace=ws, name="AzureML-AutoML")
my_env.save_to_directory('env', overwrite=True)
    


### Get scoring script and create inference config

In [22]:
best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_1548657095ae739267170c115e68972cc0d2bafad69f4b12b1a845f8e259fa99_d.txt',
 'azureml-logs/65_job_prep-tvmps_1548657095ae739267170c115e68972cc0d2bafad69f4b12b1a845f8e259fa99_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_1548657095ae739267170c115e68972cc0d2bafad69f4b12b1a845f8e259fa99_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'logs/azureml/104_azureml.log',
 'logs/azureml/azureml_automl.log',
 'logs/azureml/dataprep/python_span_11d7f95f-e4b1-47a9-ba81-f714c1453e8e.jsonl',
 'logs/azureml/dataprep/python_span_a4d74cc0-d601-4c66-b2b3-01662bcaa1d6.jsonl',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/internal_cross_validated_models.pkl',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_

In [23]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py','scoreautoml.py')

In [24]:
# create inference_config
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(entry_script="scoreautoml.py", environment = my_env)

### Deploy locally and test for testing purposes

In [25]:
deployment_config = LocalWebservice.deploy_configuration(port=9000)
service = Model.deploy(workspace=ws, name="test-service",
                       models=[model],
                       inference_config=inference_config, deployment_config=deployment_config)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_1b9697d50bbbb35eb098c299c7ed3dd0
 ---> e21868753f04
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 20897a9a560a
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjgxY2VmYWQzLWQyYzktNGY3Ny1hNDY2LTk5YTdmNTQxYzdiYiIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEzNjAwMiIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEzNjAwMiIsIndvcmtzcGFjZUlkIjoiODIxZjBlZmItMGM2MC00MGZlLTg4MDYtM2VkMDRmNDc1ZjdjIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in a13e316cf491
 ---> 8ff54cc06339
Step 4/5 : RUN mv '/var/azureml-app/tmpsa2y030t.py' /var/azureml-app/main.py
 ---> Running in 5484044bf553
 ---> fba59cb26157
Step 5/5 : 

In [26]:
#save the first two rows in a variable called tosend to use to test deployment and drop the output value
tosend = test[:2].values.tolist()
tosend = [tosend[0][:-1],tosend[1][:-1]]

In [27]:
data = json.dumps({"data":tosend})
data = bytes(data,encoding="utf8")
print(service.run(input_data=data))

Checking container health...
Local webservice is running at http://localhost:9000
{"result": [0, 0]}


In [28]:
service.delete()

Container has been successfully cleaned up.


### Deploy as a webservice

In [29]:
#set deployment_config for webservice deployment
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)

In [30]:
#deploy the model
service=Model.deploy(workspace=ws,
                    name="mywebservice",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [31]:
print(service.get_logs())

2021-01-25T15:49:21,698127134+00:00 - nginx/run 
2021-01-25T15:49:21,699655457+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-25T15:49:21,701328483+00:00 - gunicorn/run 
2021-01-25T15:49:21,701900691+00:00 - iot-server/run 
rsyslogd

### Send a request to the web service

In [32]:
url = service.scoring_uri

In [33]:
url

'http://807a442d-541f-44e9-a0d8-7c7b2db5a111.southcentralus.azurecontainer.io/score'

In [34]:
test.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
224,58.00,0,582,1,25,0,504000.00,1.00,138,1,0,205,0
100,65.00,1,305,0,25,0,298000.00,1.10,141,1,0,87,0
158,85.00,1,910,0,50,0,235000.00,1.30,134,1,0,121,0
124,60.00,0,582,0,40,0,217000.00,3.70,134,1,0,96,1
147,64.00,0,1610,0,60,0,242000.00,1.00,137,1,0,113,0


In [35]:
tosend

[[58.0, 0.0, 582.0, 1.0, 25.0, 0.0, 504000.0, 1.0, 138.0, 1.0, 0.0, 205.0],
 [65.0, 1.0, 305.0, 0.0, 25.0, 0.0, 298000.0, 1.1, 141.0, 1.0, 0.0, 87.0]]

In [36]:
#data = json.dumps({"data":tosend})
headers = {'Content-Type':'application/json'}
response = requests.post(url,data,headers=headers)
print(response.text)


"{\"result\": [0, 0]}"


### Print the logs of the webservice

In [37]:
print(service.get_logs())

2021-01-25T15:49:21,698127134+00:00 - nginx/run 
2021-01-25T15:49:21,699655457+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-25T15:49:21,701328483+00:00 - gunicorn/run 
2021-01-25T15:49:21,701900691+00:00 - iot-server/run 
rsyslogd

### Remove the webservice, models, and shut down the compute cluster

In [38]:
service.delete()

In [39]:
compute_target.delete()
